In [2]:
from PIL import Image, ImageDraw
import cv2
import numpy as np
import os, sys
import csv
import math

In [22]:
#declare img and csv files
img_file = "cellwell.png"
csv_file = "ROIPixels_2d7.csv"

In [23]:
# Read image and csv data
img = Image.open(img_file)
bbf = csv.reader(open(csv_file, newline=''), delimiter=',')

bbs = []
for row in bbf:
    if row[1] == "bb_x":
        continue
    bbs.append([int(row[1]), int(row[2]), int(row[3]), int(row[4])])

In [32]:
# Create instance segmentation mask from the coords
bbf = csv.reader(open(csv_file, newline=''), delimiter=',')
# create a segmentation mask - instance segmentation
mask = Image.new('RGB', img.size, 0)
draw = ImageDraw.Draw(mask)

for row in bbf:
    if row[1] == "bb_x":
        continue

    # get a different color for each cluster for all the rows
    color = (int(row[0])*20%255, int(row[0])*50%255, int(row[0])*100%255)

    # gather all the (x,y) points from row[5:] and mark them with a single color
    points = [int(row[i]) for i in range(5, len(row))]
    for i in range(0, len(points), 2):
        #print(points[i], points[i+1])
        draw.point((points[i], points[i+1]), fill=color)

mask.save("instance_seg.png")


In [24]:
starts = {}
def slide(img, bbs, size):
    for y0 in range(0, img.size[1], size//2):
        for x0 in range(0, img.size[0], size//2):
            x1, y1 = x0+size, y0+size
            if x1 >= img.size[0]:
                x0, x1 = img.size[0]-size, img.size[0]-1
            if y1 >= img.size[1]:
                y0, y1 = img.size[1]-size, img.size[1]-1
            # save crops
            img_crop = img.crop((x0,y0,x1,y1))
            yc=str(math.ceil(y0/(size//2)))
            xc=str(math.ceil(x0/(size//2)))
            img_crop.save("img_{yc}_{xc}.png".format(yc=yc, xc=xc))
            
            cur_bbs = []
            for bb in bbs:
                if bb[0] >= x0 and bb[0] < x1 and bb[1] >= y0 and bb[1] < y1:
                    w = bb[2] if bb[2] < size else x1-x0
                    h = bb[3] if bb[3] < size else y1-y0
                    cur_bbs.append([bb[0]-x0, bb[1]-y0, w, h])
            with open("bbs_{yc}_{xc}.csv".format(yc=yc, xc=xc), 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(cur_bbs)

In [25]:
slide(img, bbs, 512)

In [32]:
# check random image and corresponding bbox
crop, newbb = Image.open("img_12_2.png"), csv.reader(open("bbs_12_2.csv", newline=''), delimiter=',')
crop_bbs = []
for row in newbb:
    crop_bbs.append([int(row[0]), int(row[1]), int(row[2]), int(row[3])])

temp = crop.copy()
draw_crop = ImageDraw.Draw(temp)

for cbb in crop_bbs:
    draw_crop.rectangle([cbb[0], cbb[1], cbb[0]+cbb[2], cbb[1]+cbb[3]], fill=None, outline="red")
temp.show()